In [ ]:
from pathlib import Path
import numpy as np
import numpy.typing as npt
import h5py
import matplotlib.pyplot as plt
from topostats.io import LoadScans
from topostats.plotting import Colormap

colormap = Colormap()
cmap = colormap.get_cmap()
vmin = -3
vmax = 4
square_markersize_figsize_20 = 3.8

In [ ]:
base_dir = Path("/Users/sylvi/topo_data/topostats_2/datasets/20250528_RA_dose_0GY_picoz")
data_dir = base_dir / "output_fig_nodestats"
figure_dir = Path("/Users/sylvi/topo_data/topostats_2/figures/fig-disordered-tracing")
assert figure_dir.exists()
assert data_dir.exists()

image_filename = "20250528_picoz_0RA_tip_0.0_00019"

image_topostats_file_path = data_dir / "processed" / (f"{image_filename}.topostats")
assert image_topostats_file_path.exists()

loadscans = LoadScans(img_paths=[image_topostats_file_path], channel="dummy")
loadscans.get_data()
loadscans_dict = loadscans.img_dict

image_dict = loadscans_dict[image_filename]
print(image_dict.keys())
image = image_dict["image"]
p2nm = image_dict["pixel_to_nm_scaling"]
print(f"pixel to nm scaling: {p2nm} nm/pixel")
print(f"image size: {image.shape} px, {image.shape[0]*p2nm:.2f} x {image.shape[1]*p2nm:.2f} nm")

image_disordered_traces = image_dict["disordered_traces"]["above"]
image_nodestats = image_dict["nodestats"]["above"]
image_ordered_tracing = image_dict["ordered_traces"]["above"]

image_nodestats_images_grain1 = image_nodestats["images"]["grain_1"]
image_nodestats_images_grain1_grain = image_nodestats_images_grain1["grain"]
image_nodestats_images_grain1_grain_image = image_nodestats_images_grain1_grain["grain_image"]

image_disordered_traces_grain1 = image_disordered_traces["grain_1"]
image_disordered_traces_grain1_branch_indexes = image_disordered_traces_grain1["branch_indexes"]

In [ ]:
# colouring by branch index
# create a mask the the branch indexes
image_disordered_traces_grain1_branch_indexes_mask = np.ma.masked_where(
    image_disordered_traces_grain1_branch_indexes == 0,
    image_disordered_traces_grain1_branch_indexes,
)

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(image_nodestats_images_grain1_grain_image, cmap=cmap, vmin=vmin, vmax=vmax)
ax.imshow(image_disordered_traces_grain1_branch_indexes_mask, cmap="Accent", alpha=1)
plt.axis("off")
plt.savefig(figure_dir / f"i19-fig-ordered-tracing-branch-indexes-grain-1.png", bbox_inches="tight")

In [ ]:

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(image_nodestats_images_grain1_grain_image, cmap=cmap, vmin=vmin, vmax=vmax)
ordered_skeleton_mask = image_disordered_traces_grain1_branch_indexes.astype(bool)
ordered_skeleton_mask_ma = np.ma.masked_where(~ordered_skeleton_mask, ordered_skeleton_mask)
ax.imshow(ordered_skeleton_mask_ma, cmap="summer", alpha=1)

for node_id, node_image_data in image_nodestats_images_grain1["nodes"].items():
    print(f"node_id: {node_id}")
    node_branch_mask = node_image_data["node_branch_mask"]
    node_area_skeleton = node_image_data["node_area_skeleton"]
    node_avg_mask = node_image_data["node_avg_mask"]

    # plt.imshow(node_branch_mask)
    # plt.title(f"node_id: {node_id} branch mask")
    # plt.show()
    # plt.imshow(node_area_skeleton)
    # plt.title(f"node_id: {node_id} area skeleton")
    # plt.show()
    # plt.imshow(node_avg_mask)
    # plt.title(f"node_id: {node_id} avg mask")
    # plt.show()

    # make masked array for node branch mask
    node_branch_mask_ma = np.ma.masked_where(node_branch_mask == 0, node_branch_mask)
    ax.imshow(node_branch_mask_ma, cmap="cool", alpha=1)




In [ ]:
# colouring by molecule
cat_data_file = Path(
    "/Users/sylvi/topo_data/topostats_2/datasets/cats-images/output_fig_ordered_tracing_multi_molecule/processed/420231218_2ngSCcats.0_00179.topostats"
)

loadscans = LoadScans(img_paths=[cat_data_file], channel="dummy")
loadscans.get_data()
loadscans_dict = loadscans.img_dict

cat_topleft = (0, 103)

cat_image_dict = loadscans_dict[list(loadscans_dict.keys())[0]]

cat_image = cat_image_dict["image"]
cat_ordered_tracing = cat_image_dict["ordered_traces"]["above"]

cat_ordered_coords_mol0 = cat_ordered_tracing["grain_0"]["mol_0"]["ordered_coords"] + cat_topleft
cat_ordered_coords_mol1 = cat_ordered_tracing["grain_0"]["mol_1"]["ordered_coords"] + cat_topleft


fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(cat_image, cmap=cmap, vmin=vmin, vmax=vmax)
ax.plot(cat_ordered_coords_mol0[:, 1], cat_ordered_coords_mol0[:, 0], color="#2578B1")
ax.plot(cat_ordered_coords_mol1[:, 1], cat_ordered_coords_mol1[:, 0], color="#27B125")
plt.savefig(figure_dir / f"fig-ordered-tracing-molecules-cats-image", bbox_inches="tight")